# Power Products Lab

### Introduction

In this lesson, we'll practice calculating our cumulative percentages.  Let's write a query to calculate the running total of revenue brought in by product.

In [1]:
from sqlalchemy import create_engine
# change the jeffreykatz to your postgres username
conn_string = 'postgresql://jeffreykatz@localhost/ecommerce'
engine = create_engine(conn_string)

In [3]:
import pandas as pd

transactions_df = pd.read_sql('select * from transactions', engine)
transactions_df[:2]

,index,transaction_id,customer_id,product,gender,device_type,country,state,city,category,...,delivery_type,quantity,transaction_start,transaction_result,amount,individual_price,month,week,dow,hour
0,0,40170,1348959766,Hair Band,Female,Web,United States,New York,New York City,Accessories,...,one-day deliver,12,1,0,6910.0,576.0,11.0,46.0,4.0,22.0
1,1,33374,2213674919,Hair Band,Female,Web,United States,California,Los Angles,Accessories,...,one-day deliver,17,1,1,1699.0,100.0,5.0,19.0,6.0,6.0


In [9]:
import pandas as pd

query = """with  product_spend as (
    select product, sum(amount) as total_amount 
    from transactions where amount is not null
    group by product
    order by total_amount desc),
    product_running_total as (
    select product, total_amount,
    sum(total_amount) over (order by total_amount desc) as running_total,
    sum(total_amount) over () as total_spend
    from product_spend)
    select product, total_amount, running_total/total_spend as percentage_spend from product_running_total;
    """

product_percentage_spend = pd.read_sql(query, engine)
product_percentage_spend


,product,total_amount,percentage_spend
0,Fairness Cream,105504507.0,0.351122
1,Shirt,75400880.0,0.602058
2,Jean,35546772.0,0.720359
3,spectacles,25493724.0,0.805203
4,Shoes,23154294.0,0.882261
5,Books,17438460.0,0.940297
6,Bag,10069168.0,0.973807
7,watNew York Citys,7580836.0,0.999036
8,Hair Band,184642.0,0.999651
9,vessels,72061.0,0.999891


It seems that we can really just focus on a few products.

### Finding Characteristics

Next, in the cells below, go through the steps of identifying just the transactions involving the first three products.  Save those `power_product_transactions` as a table.  And then use the group by function to perform analysis on those products.